### Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
import datetime
import os
from pathlib import Path
# pip install xldr

## 0. Leitura do Ficheiro

In [ ]:
# Guardar todos os outputs numa única pasta
output_folder = Path.cwd() / "output"
output_folder.mkdir(parents=True, exist_ok=True)

# Exemplo: mapa.save(output_folder / 'mapa_de_calor_portugal_Danielle.html')

In [ ]:
p = Path.cwd()
data_folder = p.parent / "data"
shape_folder = p.parent / "shape"
file_path = data_folder / "INFsumaria_IU_2020.xls"

Sumario = pd.read_excel(file_path, header=6)

In [ ]:
Sumario = Sumario.drop(columns=Sumario.columns[[0, 1, 4, 6, 10, 11, 16]])
Sumario["Distrito"] = Sumario["Distrito"].str.title()
Sumario["Concelho"] = Sumario["Concelho"].str.title()
Sumario["Freguesia"] = Sumario["Freguesia"].str.title()
Sumario

In [ ]:
Sumario.columns

## 1.1. Coordenadas dos acidentes

## 1.2. Distritos Maps

In [ ]:
DistritoCount = Sumario["Distrito"].value_counts().reset_index()
DistritoCount

In [ ]:
file_path = shape_folder / "Distritos.shp"
Distrito = gpd.read_file(file_path)
Distrito = Distrito[(Distrito['NAME_1'] != 'Azores') & (Distrito['NAME_1'] != 'Madeira')]
# Renomeie a coluna 'NAME_2' para 'Distrito' para correspondência
Distrito.rename(columns={'NAME_1': 'Distrito'}, inplace=True)
Distrito["Distrito"] = Distrito["Distrito"].str.title()
Distrito

In [ ]:
# Mescla os dados do seu_dataframe com as geometrias dos concelhos
mapa_data = Distrito.merge(DistritoCount, how='left', left_on='Distrito', right_on='Distrito')

# Crie um mapa folium centrado em Portugal
mapa = folium.Map(location=[39.5, -8.0], zoom_start=6)

# Adicione o mapa de calor com cores diferentes para cada distrito
Choropleth(
    geo_data=mapa_data,
    name='choropleth',
    data=mapa_data,
    columns=['Distrito', 'count'],
    key_on='feature.properties.Distrito',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Acidentes'
).add_to(mapa)

# Exiba o mapa
mapa.save(output_folder / 'ANEPCdistrito.html')

## 1.2. Concelho Maps

In [ ]:
ConcelhoCount = Sumario["Concelho"].value_counts().reset_index()
ConcelhoCount.loc[ConcelhoCount['Concelho'] == 'Ponte De Sor', 'Concelho'] = 'Ponte De Sôr'
ConcelhoCount

In [ ]:
file_path = shape_folder / "concelhos.shp"
Concelho = gpd.read_file(file_path)
Concelho = Concelho[(Concelho['NAME_1'] != 'Azores') & (Concelho['NAME_1'] != 'Madeira')]
# Renomeie a coluna 'NAME_2' para 'Distrito' para correspondência
Concelho.rename(columns={'NAME_2': 'Concelho'}, inplace=True)
Concelho["Concelho"] = Concelho["Concelho"].str.title()

In [ ]:
# Obter as strings que estão em "ConcelhoCount" e não em "Concelho"
strings_nao_comuns_concelho_count = ConcelhoCount[~ConcelhoCount['Concelho'].isin(Concelho['Concelho'])]['Concelho']
print(strings_nao_comuns_concelho_count)

# Obter as strings que estão em "Concelho" e não em "ConcelhoCount"
strings_nao_comuns_concelho = Concelho[~Concelho['Concelho'].isin(ConcelhoCount['Concelho'])]['Concelho']
strings_nao_comuns_concelho

In [ ]:
Concelho[Concelho["Concelho"].str.contains("G")]

In [ ]:
# Mescla os dados do seu_dataframe com as geometrias dos concelhos
mapa_data = Concelho.merge(ConcelhoCount, how='left', left_on='Concelho', right_on='Concelho')

# Crie um mapa folium centrado em Portugal
mapa = folium.Map(location=[39.5, -8.0], zoom_start=6)

# Adicione o mapa de calor com cores diferentes para cada distrito
Choropleth(
    geo_data=mapa_data,
    name='choropleth',
    data=mapa_data,
    columns=['Concelho', 'count'],
    key_on='feature.properties.Concelho',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Acidentes'
).add_to(mapa)

# Exiba o mapa
mapa.save(output_folder / 'ANEPCconcelho.html')

## 1.3. Freguesia Maps

In [ ]:
FreguesiaCount = Sumario["Freguesia"].value_counts().reset_index()
FreguesiaCount

In [ ]:
file_path = shape_folder / "Cont_AAD_CAOP2017.shp"
Freguesia = gpd.read_file(file_path)
Freguesia["Distrito"] = Freguesia["Distrito"].str.title()
Freguesia["Concelho"] = Freguesia["Concelho"].str.title()
Freguesia["Freguesia"] = Freguesia["Freguesia"].str.title()
Freguesia

In [ ]:
# Mescla os dados do seu_dataframe com as geometrias dos concelhos
mapa_data = Freguesia.merge(FreguesiaCount, how='left', left_on='Freguesia', right_on='Freguesia')

# Crie um mapa folium centrado em Portugal
mapa = folium.Map(location=[39.5, -8.0], zoom_start=6)

# Adicione o mapa de calor com cores diferentes para cada distrito
Choropleth(
    geo_data=mapa_data,
    name='choropleth',
    data=mapa_data,
    columns=['Freguesia', 'count'],
    key_on='feature.properties.Freguesia',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Acidentes'
).add_to(mapa)

# Exiba o mapa
mapa.save(output_folder / 'ANEPCfreguesia.html')

In [ ]:
# Obter as strings que estão em "FreguesiaCount" e não em "Freguesia"
strings_nao_comuns_freguesia_count = FreguesiaCount[~FreguesiaCount['Freguesia'].isin(Freguesia['Freguesia'])]['Freguesia']
print(strings_nao_comuns_freguesia_count)

# Obter as strings que estão em "Freguesia" e não em "FreguesiaCount"
strings_nao_comuns_freguesia = Freguesia[~Freguesia['Freguesia'].isin(FreguesiaCount['Freguesia'])]['Freguesia']
strings_nao_comuns_freguesia

# Gráficos

In [ ]:
especieFogo = Sumario["Importância "].value_counts().reset_index()
especieFogo